# CMEMS WMS Data Retrieval
Construction and application of queries to retrieve data directly from Copernicus Marine Service products

_Author: Peter Kerins_  
_Created: 9 Jul 2021_  
_Environment: jupyterlab_  

Ocean Watch seeks to feature several biogeochemical parameters from the [Global Ocean Biogeochemistry Analysis and Forecast (GLOBAL_ANALYSIS_FORECAST_BIO_001_028) product](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=GLOBAL_ANALYSIS_FORECAST_BIO_001_028) of the [Copernicus Marine Service](https://marine.copernicus.eu/). The data will be mapped via Resource Watch. OW will also show time series of the parameters at various locations (representing outlets of rivers to the ocean).  

Presenting that time series data would typically require harvesting the information from the historical product rasters. However, the CMEMS [Pretty View](https://view-cmems.mercator-ocean.fr/GLOBAL_ANALYSIS_FORECAST_BIO_001_028) WebGIS tool displays just such time series (via the `Tools`). Furthermore, these are generated using the public-facing [WMS](https://www.ogc.org/standards/wms). The Copernicus Marine User Support Expert indicated [the relevant documentation](https://docs.geoserver.org/master/en/user/services/wms/reference.html#getfeatureinfo).

Constructing a properly formatted `GetFeatureInfo` request from scratch without examples was unsuccessful. However, spying on the Pretty View tool while making a time series request revealed [just such an example](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=chl&BBOX=53.26,14.26,53.2600001,14.2600001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z):

```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=chl
&BBOX=53.26,14.26,53.2600001,14.2600001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z
```

This query returns daily readings of `mass_concentration_of_chlorophyll_a_in_sea_water` at a depth of about half a meter for approximately the preceding year, at a location between Socotra and Yemen. 

A similar example closer to OW's needs can also be generated. [The query](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=o2&BBOX=31.01,31.74,31.010000100000003,31.7400001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z) for the full times series of monthly `mole_concentration_of_dissolved_molecular_oxygen_in_sea_water` values at a half-meter of depth at a pixel near the mouth of the Nile:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

This template can be modified to accommodate OW needs.

# Service

Capabilities of service requested via:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetCapabilities
```

Key excerpt from response:
```xml
            <GetFeatureInfo>
                <Format>image/png</Format>
                <Format>text/xml</Format>
                <DCPType>
                    <HTTP>
                        <Get>
                            <OnlineResource xlink:type="simple" xlink:href="http://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly" />
                        </Get>
                    </HTTP>
                </DCPType>
            </GetFeatureInfo>
```
Data can only be returned in `text/xml` format, not the preferred JSON (`application/json`).

# Parameters

## Variable
- _mole_concentration_of_dissolved_molecular_oxygen_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=o2`  
- _mole_concentration_of_phosphate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=no3`  
- _mole_concentration_of_nitrate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=po4`  
  
Note: providing a comma-separated list of valid layers (eg `QUERY_lAYERS=o2,no3`) returns an exception (`400` response)


## Depth
- `elevation=-0.49402499198913574`
- `elevation=-1.5413750410079956`
- `elevation=-2.6456689834594727`
- `elevation=-3.8194949626922607`
- `elevation=-5.078224182128906`

## Bounding Box
`xmin`,`ymin` are just the point of selection; `xmax`, `ymax` are just minimum value plus 0.0000001

# Example Request

Oxygen
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=info_format=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

 # Workflow
 Utilizing the CMEMS data entails two main steps:
 - Identify coordinates for each river mouth that overlap with the ocean water quality data products
     - Script: [river-mouths_adjust-coordinates.py](./river-mouths_adjust-coordinates.py)
     - This only needs to be executed once. The results are stored persistently 
 - For every such location, use API calls to harvest and store all relevant time series data
     - Script: [river-mouths_data-collection.py](./river-mouths_data-collection.py)

# Results

## River Mouth Locations
Original locations and effective (ie valid for WMS request) coordinates are stored in [the same table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_010_target_river_mouths), with the initially calculated location represented by the geometry, and the validated coordinates stored in `x_valid` and `y_valid`. 

In [3]:
import os
from dotenv import dotenv_values
env_vars = dotenv_values('/home/pkerins/code/.env')
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import pandas as pd
CARTO_USER = env_vars['CARTO_WRI_RW_USER']
CARTO_KEY = env_vars['CARTO_WRI_RW_KEY']
set_default_credentials(username=CARTO_USER,
                        base_url="https://{user}.carto.com/".format(user=CARTO_USER),
                        api_key=CARTO_KEY)

In [4]:
read_carto('ocn_calcs_010test_target_river_mouths').tail()

,cartodb_id,the_geom,main_riv,distance,upland_skm,ord_clas,ord_stra,catch_skm,length_km,dist_dn_km,ord_flow,dist_up_km,next_down,hyriv_id,dis_av_cms,endorheic,hybas_l12,x_valid,y_valid
5724,5725,POINT (-72.97455 -43.77369),61575734,850.486585,13161.8,1,7,6.35,4.49,0.0,4,331.7,0,61575734,985.256,0,6120025410,-73.1745487654631,-43.773688525069524
5725,5726,POINT (-68.55295 -49.93353),61603671,1856.590403,18118.1,1,5,4.29,2.49,0.0,5,491.1,0,61603671,46.723,0,6120021180,-68.34082117662179,-50.145658427464056
5726,5727,POINT (-68.96083 -51.59819),61608832,41578.196873,10478.0,1,5,34.91,5.60,0.0,5,302.8,0,61608832,80.302,0,6120021500,-68.9608316434375,-51.59819353927945
5727,5728,POINT (-68.57559 -49.95131),61603772,3688.239087,26959.0,1,6,5.52,1.49,0.0,4,588.1,0,61603772,710.555,0,6120021210,-68.36345557356908,-50.163445781566246
5728,5729,POINT (-73.25994 -52.73331),61614285,3057.544015,762.8,1,5,0.65,1.08,0.0,4,62.5,0,61614285,114.223,0,6120022650,-73.47207664454277,-52.94543879690165


## Chemical Concentrations at River Mouths
Corrected locations are used to retrieve longitudinal data for the concentration of various chemicals at a range of depths. The results for all locations, chemicals, and depths are stored in [a single table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_011_river_mouth_chemical_concentrations).  

In [ ]:
read_carto('ocn_calcs_011_river_mouth_chemical_concentrations').tail()